In [189]:
class Node:
    def __init__(self,char,freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None
    def __repr__(self):
        return f"Node({self.char},{self.freq})"
    def __lt__(self,other):
        return self.freq < other.freq

In [190]:
freqdict = {}

with open('sample.txt','r') as file:
    text = file.read()
  
print(text)

for char in text:
    if char not in freqdict.keys():
        freqdict[char] = 0
    freqdict[char]+=1
freqdict

BCCABBDDAECCBBAEDDCC


{'B': 5, 'C': 6, 'A': 3, 'D': 4, 'E': 2}

In [191]:
h = []
for char,freq in freqdict.items():
    heappush(h,(freq,Node(char,freq)))

In [192]:
h

[(2, Node(E,2)),
 (3, Node(A,3)),
 (5, Node(B,5)),
 (6, Node(C,6)),
 (4, Node(D,4))]

In [193]:
while(len(h)>1):
    node1 = heappop(h)[1]
    node2 = heappop(h)[1]
    sumval = node1.freq  + node2.freq
    print(sumval)
    treenode = Node(None,sumval)
    treenode.left = node1
    treenode.right = node2
    heappush(h,(sumval,treenode))
    

5
9
11
20


In [194]:
top = h[0][1]
codebook = {}
def traversetree(tree,code):
    if not tree:
        return
    if tree.char:
        codebook[tree.char] = code
    else:
        traversetree(tree.left,code+'0')
        traversetree(tree.right,code+'1')

traversetree(top,'')

In [195]:
codebook

{'D': '00', 'B': '01', 'E': '100', 'A': '101', 'C': '11'}